# Catalog Design Pattern

A Class that holds a dict of static methods that can be triggered based on a key/param



In [12]:
class Catalog:
    def __init__(self, param):
        self._static_method_choices = {
            'param_value_1': self._static_method_1,
            'param_value_2': self._static_method_2,
            'param_value_3': self._static_method_3,            
        }
    
        if param in self._static_method_choices.keys():
            self.param = param
        else:
            raise ValueError(f"Invalid Value for Param: {param}")
    
    @staticmethod
    def _static_method_1():
        print("Executed Method 1!")
    
    @staticmethod
    def _static_method_2():
        print("Executed Method 2!")
        
    @staticmethod
    def _static_method_3():
        print("Executed Method 3!")
        
    def main_method(self):
        self._static_method_choices[self.param]()
        
        
class CatalogInstance:
    def __init__(self, param):
        self.x1 = 'x1'
        self.x2 = 'x2'
        
        if param in self._instance_method_choices:
            self.param = param
            
        else:
            raise ValueError(f"Invalid Value for Param: {param}")
    
    def _instance_method_1(self):
        print(f"Value {self.x1}")
    
    def _instance_method_2(self):
        print(f"Value {self.x2}")
    
    _instance_method_choices = {
        'param_value_1': _instance_method_1,
        'param_value_2': _instance_method_2
    }
    
    def main(self):
        self._instance_method_choices[self.param].__get__(self)()
        
        
class CatalogClass:
    x1 = 'x1'
    x2 = 'x2'
    
    def __init__(self, param):
        if param in self._class_method_choices.keys():
            self.param = param
        else:
            raise ValueError(f"Invalid Value for Param: {param}")
    
    @classmethod
    def _class_method_1(cls):
        print(f"Value {cls.x1}")
    
    @classmethod
    def _class_method_2(cls):
        print(f"Value {cls.x2}")
    
    _class_method_choices = {
        'param_value_1': _class_method_1,
        'param_value_2': _class_method_2,
    }
    
    def main(self):
        self._class_method_choices[self.param].__get__(None, self.__class__)()
        

class CatalogStatic:
    def __init__(self, param):
        if param in self._static_method_choices.keys():
            self.param = param
        else:
            raise ValueError(f"Invalid value for Param: {param}")
        
    @staticmethod
    def _static_method_1():
        print("executed method 1!")

    @staticmethod
    def _static_method_2():
        print("executed method 2!")

    _static_method_choices = {'param_value_1': _static_method_1, 'param_value_2': _static_method_2}

    def main(self):
        self._static_method_choices[self.param].__get__(None, self.__class__)()
        


In [17]:
def main():
    test0 = Catalog('param_value_2')
    test0.main_method()
    
    test1 = Catalog('param_value_1')
    test1.main_method()
    
    test2 = CatalogInstance('param_value_1')
    test2.main()
    
    test3 = CatalogInstance('param_value_2')
    test3.main()
    
    test4 = CatalogStatic('param_value_1')
    test4.main()
    
    test5 = CatalogStatic('param_value_2')
main()

Executed Method 2!
Executed Method 1!
Value x1
Value x2
executed method 1!
